In [ ]:
#USER INPUTS BElOW

#Number of splines to be constructed
n_voronoi = 10
#Number of pixels per axis
division =15
#Noise for grey image construction
noise = 5

#Creating a pixel space
import numpy as np
import random
import plotly.graph_objects as go

x = np.linspace(0, 1, division)
y = np.linspace(0, 1, division)
z = np.linspace(0, 1, division)
mesh = np.meshgrid(x, y, z)
pixels = list(zip(*(dim.flat for dim in mesh)))


#Indexing pixels (ipixels); so that each pixel can be accessed with x,y, and z co-ordinates
ipixels = np.array(pixels)
ipixels = ipixels.reshape(division**2,division*3)
ipixels = np.transpose(ipixels)
ipixels = ipixels.reshape(division*3, division,division)
ipixels = np.transpose(ipixels)
ipixels = ipixels.reshape(division,division,division,3)


#Finding all the 26 neighbours of a pixel and the pixel itself; returns 27 points
def find_neighbours(i,j,k):  
    neighbours = []
    import itertools
    I = []
    J = []
    K = []
    for p in range(i-1,i+2):
        if p >= 0 and p < division:
            I.append(p)
    for p in range(j-1,j+2):
        if p >= 0 and p < division:
            J.append(p)
    for p in range(k-1,k+2):
        if p >= 0 and p < division:
            K.append(p)

    neighbours_index =list(itertools.product(I,J,K))
#     neighbours_index.remove((i,j,k))

#     for neighbour_index in neighbours_index:
#         neighbours.append(ipixels[neighbour_index[0]][neighbour_index[1]][neighbour_index[2]])

    return neighbours_index

#Findind the neighbours along each axis; for a 3d pixel - 6 neighbours and the actual point; returns 7 points
def find_Neighbours(i,j,k):
    I = []
    J = []
    K = []
    neighbours = []
    
    for p in [i-1,i+1]:
        if p >= 0 and p < division:
            I.append(p)
    for p in [j-1,j+1]:
        if p >= 0 and p < division:
            J.append(p)
    for p in [k-1,k+1]:
        if p >= 0 and p < division:
            K.append(p)
    for m in range(len(I)):
        neighbours.append((I[m],j,k))
    for m in range(len(J)):
        neighbours.append((i,J[m],k))
    for m in range(len(K)):
        neighbours.append((i,j,K[m]))
        
    neighbours.append((i,j,k))
    return neighbours

#Seeds determine how many Voronoi cells should be there in total; each seed will have 3 co-ordinates insode the unit cube
seeds = np.random.rand(n_voronoi,3)

#Defining the voronoi cells; mathematical defenition; There will be same number of voronoi cells as there are seeds
vorpixel = [[] for _ in range(seeds.shape[0])]

for i in range(division):
    for j in range(division):
        for k in range(division):

            dist_list = []
            for seed in seeds:
                dist = np.linalg.norm(seed-ipixels[i][j][k])
                dist_list.append(dist)
            index = dist_list.index(min(dist_list))
            vorpixel[index].append((i,j,k))
    



#Plotting Voronoi cells unit cube
fig = go.Figure()
for i in range(len(vorpixel)):
    print_pix = []
    for index in vorpixel[i]:
        print_pix.append(ipixels[index])
    X, Y, Z = np.array(print_pix)[:,0], np.array(print_pix)[:,1], np.array(print_pix)[:,2]
    fig.add_trace(go.Scatter3d(x=X,y=Y,z=Z, marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig7.pdf")

#Contructing the totpixels without indexing but with co-ordinates and not with actual values like 'pixels'
x = np.arange(0, division)
y = np.arange(0, division)
z = np.arange(0, division)
mesh = np.meshgrid(x, y, z)
totpixels = list(zip(*(dim.flat for dim in mesh)))


#Boundary between the voronoi cells. Boundary between two vornoi cells will have the last layer of pixels from both 

membrane = []

for pix in totpixels:
    Neighbours = find_Neighbours(pix[0],pix[1],pix[2])

    c=0
    for pixel in vorpixel:
        c = c + set(Neighbours).issubset(set(pixel))
    if c==0:
        membrane.append(pix)


#Plotting the membrane
surface =[]
for point in membrane:
    surface.append(ipixels[point])
surface = np.array(surface)
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=surface[:,0],y=surface[:,1],z=surface[:,2], marker = dict(size=2), mode='markers'))
fig.update_layout(
    scene = dict(
        xaxis = dict( range=[0,1],),
        yaxis = dict( range=[0,1],),
        zaxis = dict( range=[0,1],)))
fig.show()


#Merging Volumes until the number of volumes equals the choice; Merging only takes place if two cells are adjacent
import random

choice = 4
mergedVol = vorpixel

while len(mergedVol) != choice:

    molecule = random.choice(membrane)

    Nbors = find_Neighbours(molecule[0], molecule[1], molecule[2])
    indexlist = []
    for nbor in Nbors:
        for z in range(len(mergedVol)):
            [indexlist.append(z) for item in mergedVol[z] if item==nbor]

    #The index list now has the indices of the neighbours of a random point from the membrane

    mergeA = indexlist.pop(-1)

    mergeB = np.random.choice(indexlist)

    #Merging the two volumes
    if mergeA != mergeB:
        
        mergedVol_temp = []
        for i in range(len(mergedVol)):
            if (i != mergeA and i != mergeB):
                mergedVol_temp.append(mergedVol[i])

        mergedVol_temp.append(mergedVol[mergeA] + mergedVol[mergeB])

        mergedVol = mergedVol_temp


#Plotting the Voronoi cells after merging
fig = go.Figure()
for i in range(len(mergedVol)):
    print_pix = []
    for index in mergedVol[i]:
        print_pix.append(ipixels[index])
    X, Y, Z = np.array(print_pix)[:,0], np.array(print_pix)[:,1], np.array(print_pix)[:,2]
    fig.add_trace(go.Scatter3d(x=X,y=Y,z=Z, marker = dict(size=2), mode='markers'))
fig.show()

'''Imaging the membrane by assigning intensities to pixels based on their closeness to the membrane;
Inensities for background and foreground are sampled from two normal distributions and the choice of the normal dist. is 
made based on probabilities defined by closeness to membrane'''

grey_pixels = []
theta_1 = -55
theta_2 = 1100

for pixel in pixels:
    s_dist = 1.73
    
    for molecule in membrane:
        
        dist = np.linalg.norm(pixel-ipixels[molecule])
        if dist < s_dist:
            s_dist = dist
   
    p = 1/(1+np.exp(-1*(theta_1+theta_2*s_dist)))
    N_c = np.abs(np.random.normal(0,noise))
    N_b = 255 - np.abs(np.random.normal(0,noise))
    Ndist_list = [N_c,N_b]
    
    grey_pixels.append(list(pixel))
    
    grey_value = random.choices(Ndist_list, weights=(1-p,p), k=1)
    
    grey_pixels[-1].append(grey_value[0])

#Volume image based on assigned pixel intensity
grey_pixels = np.array(grey_pixels)

fig = go.Figure(data=go.Volume(
    x=grey_pixels[:,0],
    y=grey_pixels[:,1],
    z=grey_pixels[:,2],
    value=grey_pixels[:,3],
    isomin=0,
    isomax=255,
    opacity=0.02, # needs to be small to see through all surfaces
    surface_count=25, # needs to be a large number for good volume rendering
    ))
#fig.update_layout(width =1000, height =1000)
fig.show()


#Indexing grey_pixels (igrey_pixels); so that each pixel can be accessed with x,y, and z co-ordinates
igrey_pixels = grey_pixels.reshape(division, division, division, 4)
igrey_pixels = igrey_pixels.reshape(division**2,division*4)
igrey_pixels = np.transpose(igrey_pixels)
igrey_pixels = igrey_pixels.reshape(division*4, division,division)
igrey_pixels = np.transpose(igrey_pixels)
igrey_pixels = igrey_pixels.reshape(division,division,division,4)

#Finding neigbours without returning the actual point - just 6 neighbours
def find_Neighbours(i,j,k):
    I = []
    J = []
    K = []
    neighbours = []
    
    for p in [i-1,i+1]:
        if p >= 0 and p < division:
            I.append(p)
    for p in [j-1,j+1]:
        if p >= 0 and p < division:
            J.append(p)
    for p in [k-1,k+1]:
        if p >= 0 and p < division:
            K.append(p)
    for m in range(len(I)):
        neighbours.append((I[m],j,k))
    for m in range(len(J)):
        neighbours.append((i,J[m],k))
    for m in range(len(K)):
        neighbours.append((i,j,K[m]))
        
#     neighbours.append((i,j,k))
    return neighbours



#### Seeded Region Growing algorithm

In [ ]:
#Finds regions along with the membrane from the igrey_pixels i.e., after imaging 
import random 

All_regions = []
totpixels_copy = totpixels.copy()

while len(totpixels_copy) > 0:
    junk = []
    region = [random.choice(totpixels_copy)]
    subregion = region
    while len(subregion) != 0:
        summ = 0
        for x in range(len(region)):
            summ = summ + igrey_pixels[region[x]][3]
        mean = summ/len(region)
        n=0
        for pixel in subregion:

            Neighbours  = find_Neighbours(pixel[0],pixel[1],pixel[2])
            Neighbours = list(set(Neighbours).difference(junk))
            junk.extend(Neighbours)
            junk.append(pixel)
            junk = list(set(junk))
            for neighbour in Neighbours:
                if np.abs(igrey_pixels[neighbour][3] - mean) < 20:
                    region.append(neighbour)
                    n = n+1

        if n==0:
            subregion = []
        else:
            subregion = region[-n:]

    All_regions.append(region)
    totpixels_copy = list(set(totpixels_copy).difference(region))

len(All_regions)

#Plotting the output of the region growing algorithm
fig = go.Figure()

for i in range(0,len(All_regions)):
    segplot = []
    for reg in All_regions[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1],z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig10.pdf")

#Cross sectional slice 

cross_section = [x for x in totpixels if x[0]== int(division/2)]
temp_cross = []
for i in range(0,len(All_regions)):
    temp_cross.append(list(set(cross_section).intersection(All_regions[i])))
cross = [x for x in temp_cross if x != []]


fig = go.Figure()

for i in range(0,len(cross)):
    segplot = []
    for reg in cross[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1], z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig11a.pdf")

#Cross sectional slice 

cross_section = [x for x in totpixels if x[1]== int(division/2)]
temp_cross = []
for i in range(0,len(All_regions)):
    temp_cross.append(list(set(cross_section).intersection(All_regions[i])))
cross = [x for x in temp_cross if x != []]


fig = go.Figure()

for i in range(0,len(cross)):
    segplot = []
    for reg in cross[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1], z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig11b.pdf")

#Cross sectional slice 

cross_section = [x for x in totpixels if x[2]== int(division/2)]
temp_cross = []
for i in range(0,len(All_regions)):
    temp_cross.append(list(set(cross_section).intersection(All_regions[i])))
cross = [x for x in temp_cross if x != []]


fig = go.Figure()

for i in range(0,len(cross)):
    segplot = []
    for reg in cross[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1], z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig11c.pdf")

#Getting all pairs of pixels from the total pixel space
import itertools
Pairs = set()

for subset in itertools.combinations(totpixels, 2):
    Pairs.add(frozenset(subset))

# background = list(set(totpixels).difference(membrane))
# Truths = [background,membrane]

# #Calculating Rand's Index conventional logic
# TP = TN = FN = FP = 0

# for pair in Pairs:
    
#     org = 0 
#     for truth in vorpixel:
#         org  = org + set(pair).issubset(truth)
        
#     seg = 0 
#     for allregion in All_regions:
#         seg  = seg + set(pair).issubset(allregion)
        
#     if org == 1 and seg == 1:
#         TP = TP + 1
        
#     elif org == 0 and seg == 0:
#         TN = TN + 1
        
#     elif org == 1 and seg == 0:
#         FN = FN + 1
    
#     elif org == 0 and seg == 1:
#         FP = FP + 1
        
# R_index = (TP+TN)/(TP+TN+FN+FP)   
# print(R_index)



#Alternate Rand's index calculation for computation time optimization

X_pairs = set()
Y_pairs = set()

for vor in vorpixel:

    dopplets = set()

    for dopplet in itertools.combinations(vor, 2):
        dopplets.add(frozenset(dopplet))

    X_pairs.update(dopplets)

for allregion in All_regions:

    dopplets = set()

    for dopplet in itertools.combinations(allregion, 2):
        dopplets.add(frozenset(dopplet))

    Y_pairs.update(dopplets)
    

TP = len(X_pairs.intersection(Y_pairs))
TN = len(Pairs) - len(X_pairs.union(Y_pairs))
FP = len(X_pairs.difference(Y_pairs))
FN = len(Y_pairs.difference(X_pairs))

R_index = (TP+TN)/(TP+TN+FN+FP) 

print("Rand Index: ",R_index)

#Calculating Variation of Information
n =division**3
VI = 0
p = []
q = []
r = np.array([[None]*len(All_regions)]*len(vorpixel))

for i in range(len(vorpixel)):
    p.append(len(vorpixel[i])/n)
    
for j in range(len(All_regions)):
    q.append(len(All_regions[j])/n)

for i in range(len(vorpixel)):
    for j in range(len(All_regions)):
        r[i][j] = len(set(vorpixel[i]).intersection(set(All_regions[j])))/n
        
        
for i in range(len(vorpixel)):
    for j in range(len(All_regions)):
        if r[i][j]!=0:
            VI = VI + r[i][j]*(np.log(r[i][j]/p[i]) + np.log(r[i][j]/q[j]))

VI = -1*VI
        

print("Variation of Information: ",VI)

# Neigh_pairs = []
# for pair in Pairs:
#     C = np.abs([a - b for a, b in zip(pair[0], pair[1])]).sum()
#     if C==1:
#         Neigh_pairs.append(pair)


#### Greedy Additive Edge Contraction Algorithm

In [ ]:

#Getting all pairs of pixels from the total pixel space for GAEC non frozen set
import itertools
Pairs = []

for subset in itertools.combinations(totpixels, 2):
    Pairs.append(subset)

'''Defining costs for edges based on the pixel intensity difference between the two pixels contributing to the edge.
Negative costs for edges having high difference as negative edges should be cut. And we are looking to minimize the 
function'''

cost_Edash = []
cost_E = []
Neigh_pairs = []

for pair in Pairs:
    chi = np.abs(igrey_pixels[pair[0]][3]-igrey_pixels[pair[1]][3])
    C = np.abs([a - b for a, b in zip(pair[0], pair[1])]).sum()
    if chi>20:
        chi = -1*chi
        
    temp = list([set([pair[0]]),set([pair[1]])])
    temp.append(chi)
    
    C = np.abs([a - b for a, b in zip(pair[0], pair[1])]).sum()
    if C==1:
        Neigh_pairs.append(pair)
        cost_E.append(temp)
    
    cost_Edash.append(temp)
    
    
#Example test sets commented below

# cost_E = [[{'a'},{'b'},-1],[{'b'},{'c'},2],[{'c'},{'d'},17],[{'d'},{'e'},-5],[{'e'},{'f'},2],[{'f'},{'a'},0],[{'b'},{'e'},3]]
# cost_Edash = [[{'a'},{'b'},-1],[{'b'},{'c'},2],[{'c'},{'d'},17],[{'d'},{'e'},-5],[{'e'},{'f'},2],[{'f'},{'a'},0],[{'b'},{'e'},3],
#               [{'a'},{'e'},1],[{'b'},{'f'},-4],[{'b'},{'d'},3],[{'e'},{'c'},6],[{'f'},{'d'},-20],[{'a'},{'c'},7],
#               [{'f'},{'c'},5],[{'a'},{'d'},0]]

# cost_E = [[{'a'},{'b'},20],[{'b'},{'c'},17],[{'c'},{'d'},-18],[{'d'},{'a'},-3]]
# cost_Edash = [[{'a'},{'b'},20],[{'b'},{'c'},17],[{'c'},{'d'},-18],[{'d'},{'a'},-3], [{'a'},{'c'},-1],[{'b'},{'d'},30]]

# cost_E = [[{'a'},{'b'},10],[{'b'},{'c'},-5],[{'c'},{'d'},11],[{'d'},{'a'},-2]]
# cost_Edash = [[{'a'},{'b'},10],[{'b'},{'c'},-5],[{'c'},{'d'},11],[{'d'},{'a'},-2], [{'a'},{'c'},0],[{'b'},{'d'},1]]

#Finding the current maximum cost
def Maximum(costs):
    large = -2147483648
    for cost in costs:
        if cost[-1]>large:
            large = cost[-1]
            edge = cost
    return edge, large

#During edge contraction adding the costs of duplicate edges with different costs
def same_edge_sum(testing):    
    final = []
    while len(testing)!=0:

        item1 = testing.pop(0)
        for item2 in testing:
            if set(frozenset(i) for i in item1[:2]) == set(frozenset(i) for i in item2[:2]):
                
                item1[2] = item1[2]+item2[2]
                testing.remove(item2)
                
        final.append(item1)

#     final.extend(testing)
    return final

#Contract the edge and return the unchanged and changed edges separately
def edge_contract(costs, edge):
    
#     costs.remove(edge)
    costs = [value for value in costs if value[:2] != edge[:2]]
    indices = []
    
    r = edge[0].union(edge[1])
    for cost in costs:
        
        if edge[0] in cost:
            indices.append(costs.index(cost))
            cost[cost.index(edge[0])] = r
            
        if edge[1] in cost:
            indices.append(costs.index(cost))
            cost[cost.index(edge[1])] = r
        
        
    q =[costs[i] for i in indices]
    
    costs = [x for x in costs if x not in q]
    
    edge_addition = same_edge_sum(q)
#     costs.extend(edge_addition)

    
    return costs, edge_addition
    

#GAEC Algorithm

while len(cost_E)!=0:
    
    ab, X_ab = Maximum(cost_E)
    if X_ab < 0:
        break
        
    cost_E, cost_E_edges = edge_contract(cost_E, ab)
    cost_Edash, cost_Edash_edges = edge_contract(cost_Edash, ab)
    
#Adjusting cost_E according to the updated costs from cost_Edash
    for item1 in cost_E_edges:
        for item2 in cost_Edash_edges:
            if set(frozenset(i) for i in item1[:2]) == set(frozenset(i) for i in item2[:2]):
                item1[2] = item2[2]
                
    
    cost_E.extend(cost_E_edges)
    cost_Edash.extend(cost_Edash_edges)
            

#Unraveling the sets to separate regions in order to plot later
decomposed = []
for cost in cost_E:
    decomposed.extend(cost[:2])
Decom = list(set(frozenset(item) for item in decomposed)) 

#Plotting the decomposed regions
fig = go.Figure()

for set_ in Decom:
    segplot = []
    for reg in list(set_):
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1],z=segplot[:,2], marker = dict(size=2), mode='markers'))

fig.show()

#Calculating Rand's Index
TP = TN = FN = FP = 0

for pair in Pairs:
    
    org = 0 
    for truth in vorpixel:
        org  = org + set(pair).issubset(truth)
        
    seg = 0 
    for allregion in Decom:
        seg  = seg + set(pair).issubset(allregion)
        
    if org == 1 and seg == 1:
        TP = TP + 1
        
    elif org == 0 and seg == 0:
        TN = TN + 1
        
    elif org == 1 and seg == 0:
        FN = FN + 1
    
    elif org == 0 and seg == 1:
        FP = FP + 1
        
R_index = (TP+TN)/(TP+TN+FN+FP)   

print("Rand Index: ",R_index)

# def intersection(list1, list2):
    
#     junk1 = [item[:2] for item in list2]
#     list3 = [value for value in list1 if value[:2] in junk1]
    
#     junk2 = [item[:2] for item in list3]
#     list4 = [val for val in list2 if val[:2] not in junk2 ]
#     list5 = list3 + list4
    
#     return list5

#Calculating Variation of Information
n =division**3
VI = 0
p = []
q = []
r = np.array([[None]*len(Decom)]*len(vorpixel))

for i in range(len(vorpixel)):
    p.append(len(vorpixel[i])/n)
    
for j in range(len(Decom)):
    q.append(len(Decom[j])/n)

for i in range(len(vorpixel)):
    for j in range(len(Decom)):
        r[i][j] = len(set(vorpixel[i]).intersection(set(Decom[j])))/n
        
        
for i in range(len(vorpixel)):
    for j in range(len(Decom)):
        if r[i][j]!=0:
            VI = VI + r[i][j]*(np.log(r[i][j]/p[i]) + np.log(r[i][j]/q[j]))

VI = -1*VI
        

print("Variation of Information: ",VI)
